# Final Project Part 2
## Author: Moe LaGrand

In [2]:
import pandas as pd
import numpy as np
import sqlite3

### Import Data

In [3]:
df_train = pd.read_csv('data/train.csv')

In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Prep Data

In [5]:
# convert ages to ints
df_train['Age'] = df_train['Age'].astype("int32", errors="ignore")

In [6]:
# convert fare to 2 decimal place float
df_train['Fare'] = df_train['Fare'].round(2)

In [7]:
# encoding male and female to binary values for classification purposes
df_train.loc[df_train['Sex'] == "male", "Sex"] = 0
df_train.loc[df_train['Sex'] == "female", "Sex"] = 1

In [8]:
ordered_df = df_train[['PassengerId','Name','Sex','Age','Survived','Pclass','SibSp','Parch','Ticket','Fare','Embarked']]

## Machine Learning Stuff

In [9]:
ordered_df.head()

,PassengerId,Name,Sex,Age,Survived,Pclass,SibSp,Parch,Ticket,Fare,Embarked
0,1,"Braund, Mr. Owen Harris",0,22.0,0,3,1,0,A/5 21171,7.25,S
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,1,1,0,PC 17599,71.28,C
2,3,"Heikkinen, Miss. Laina",1,26.0,1,3,0,0,STON/O2. 3101282,7.92,S
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,1,1,0,113803,53.10,S
4,5,"Allen, Mr. William Henry",0,35.0,0,3,0,0,373450,8.05,S


In [10]:
features = ['Sex', 'Age', 'Pclass', 'SibSp', 'Parch']

In [12]:
ordered_df[features].dropna()

,Sex,Age,Pclass,SibSp,Parch
0,0,22.0,3,1,0
1,1,38.0,1,1,0
2,1,26.0,3,0,0
3,1,35.0,1,1,0
4,0,35.0,3,0,0
...,...,...,...,...,...
885,1,39.0,3,0,5
886,0,27.0,2,0,0
887,1,19.0,1,0,0
889,0,26.0,1,0,0


### Load Data into DB

In [13]:
conn = sqlite3.connect('titanic')  
c = conn.cursor()

#### Pclass

In [57]:
#c.execute('DROP TABLE Pclass;')

In [58]:
c.execute('''
    CREATE TABLE Pclass(
        pclass_id INT PRIMARY KEY,
        class VARCHAR(10)
    )
''')

In [59]:
classes = [[1,"upper"], [2,"middle"], [3,"lower"]]

In [60]:
c.executemany("INSERT INTO Pclass VALUES (?, ?)", classes)

In [61]:
for row in c.execute('''
select * from Pclass
'''):
    print(row)

(1, 'upper')
(2, 'middle')
(3, 'lower')


#### Port Table

In [69]:
#c.execute('DROP TABLE Port;')

In [70]:
c.execute('''
    CREATE TABLE Port(
        port_id VARCHAR(1) PRIMARY KEY,
        name VARCHAR(20)
    )
''')

In [71]:
ports = [["C","Cherbough"], ["Q","Queenstown"], ["S","Southhampton"]]

In [72]:
c.executemany("INSERT INTO Port VALUES (?, ?)", ports)

In [73]:
for row in c.execute('''
select * from Port
'''):
    print(row)

('C', 'Cherbough')
('Q', 'Queenstown')
('S', 'Southhampton')


#### Passenger Table

In [78]:
#c.execute('DROP TABLE Passenger;')

In [75]:
c.execute('''
    CREATE TABLE Passenger(
        passenger_id INT PRIMARY KEY,
        name VARCHAR(50),
        sex BINARY,
        age INT,
        survived BINARY,
        pclass_id INT,
        sibsp INT,
        parch INT,
        ticket VARCHAR(50),
        fare FLOAT(2),
        port_id VARCHAR(1),
        FOREIGN KEY (pclass_id) REFERENCES Pclass(pclass_id),
        FOREIGN KEY (port_id) REFERENCES Port(port_id)
    )
''')

In [76]:
c.executemany("INSERT INTO Passenger VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", ordered_df.values)

In [77]:
for row in c.execute('''
select * from Passenger
'''):
    print(row)

(1, 'Braund, Mr. Owen Harris', 0, 22, 0, 3, 1, 0, 'A/5 21171', 7.25, 'S')
(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 1, 38, 1, 1, 1, 0, 'PC 17599', 71.28, 'C')
(3, 'Heikkinen, Miss. Laina', 1, 26, 1, 3, 0, 0, 'STON/O2. 3101282', 7.92, 'S')
(4, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 1, 35, 1, 1, 1, 0, '113803', 53.1, 'S')
(5, 'Allen, Mr. William Henry', 0, 35, 0, 3, 0, 0, '373450', 8.05, 'S')
(6, 'Moran, Mr. James', 0, None, 0, 3, 0, 0, '330877', 8.46, 'Q')
(7, 'McCarthy, Mr. Timothy J', 0, 54, 0, 1, 0, 0, '17463', 51.86, 'S')
(8, 'Palsson, Master. Gosta Leonard', 0, 2, 0, 3, 3, 1, '349909', 21.08, 'S')
(9, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 1, 27, 1, 3, 0, 2, '347742', 11.13, 'S')
(10, 'Nasser, Mrs. Nicholas (Adele Achem)', 1, 14, 1, 2, 1, 0, '237736', 30.07, 'C')
(11, 'Sandstrom, Miss. Marguerite Rut', 1, 4, 1, 3, 1, 1, 'PP 9549', 16.7, 'S')
(12, 'Bonnell, Miss. Elizabeth', 1, 58, 1, 1, 0, 0, '113783', 26.55, 'S')
(13, 'Saundercock, Mr. Willi

### Useful Queries

#### Get All Passengers

In [80]:
def get_all_passengers():
    for row in c.execute('''
        select * from Passenger
        '''):
        print(row)

In [81]:
get_all_passengers()

(1, 'Braund, Mr. Owen Harris', 0, 22, 0, 3, 1, 0, 'A/5 21171', 7.25, 'S')
(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 1, 38, 1, 1, 1, 0, 'PC 17599', 71.28, 'C')
(3, 'Heikkinen, Miss. Laina', 1, 26, 1, 3, 0, 0, 'STON/O2. 3101282', 7.92, 'S')
(4, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 1, 35, 1, 1, 1, 0, '113803', 53.1, 'S')
(5, 'Allen, Mr. William Henry', 0, 35, 0, 3, 0, 0, '373450', 8.05, 'S')
(6, 'Moran, Mr. James', 0, None, 0, 3, 0, 0, '330877', 8.46, 'Q')
(7, 'McCarthy, Mr. Timothy J', 0, 54, 0, 1, 0, 0, '17463', 51.86, 'S')
(8, 'Palsson, Master. Gosta Leonard', 0, 2, 0, 3, 3, 1, '349909', 21.08, 'S')
(9, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 1, 27, 1, 3, 0, 2, '347742', 11.13, 'S')
(10, 'Nasser, Mrs. Nicholas (Adele Achem)', 1, 14, 1, 2, 1, 0, '237736', 30.07, 'C')
(11, 'Sandstrom, Miss. Marguerite Rut', 1, 4, 1, 3, 1, 1, 'PP 9549', 16.7, 'S')
(12, 'Bonnell, Miss. Elizabeth', 1, 58, 1, 1, 0, 0, '113783', 26.55, 'S')
(13, 'Saundercock, Mr. Willi

#### Restrict by class

In [101]:
def get_by_class(classes):
    clause = "WHERE "
    while len(classes) > 0:
        cls = classes.pop()
        clause += f"pc.class = '{cls}'"
        if len(classes) > 0:
            clause += "OR "
        
    for row in c.execute(f'''
        SELECT *
        FROM Passenger AS ps
        JOIN Pclass AS pc ON ps.pclass_id = pc.pclass_id
        {clause}
        '''):
            print(row)

In [104]:
get_by_class(["upper"])

(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 1, 38, 1, 1, 1, 0, 'PC 17599', 71.28, 'C', 1, 'upper')
(4, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 1, 35, 1, 1, 1, 0, '113803', 53.1, 'S', 1, 'upper')
(7, 'McCarthy, Mr. Timothy J', 0, 54, 0, 1, 0, 0, '17463', 51.86, 'S', 1, 'upper')
(12, 'Bonnell, Miss. Elizabeth', 1, 58, 1, 1, 0, 0, '113783', 26.55, 'S', 1, 'upper')
(24, 'Sloper, Mr. William Thompson', 0, 28, 1, 1, 0, 0, '113788', 35.5, 'S', 1, 'upper')
(28, 'Fortune, Mr. Charles Alexander', 0, 19, 0, 1, 3, 2, '19950', 263.0, 'S', 1, 'upper')
(31, 'Uruchurtu, Don. Manuel E', 0, 40, 0, 1, 0, 0, 'PC 17601', 27.72, 'C', 1, 'upper')
(32, 'Spencer, Mrs. William Augustus (Marie Eugenie)', 1, None, 1, 1, 1, 0, 'PC 17569', 146.52, 'C', 1, 'upper')
(35, 'Meyer, Mr. Edgar Joseph', 0, 28, 0, 1, 1, 0, 'PC 17604', 82.17, 'C', 1, 'upper')
(36, 'Holverson, Mr. Alexander Oskar', 0, 42, 0, 1, 1, 0, '113789', 52.0, 'S', 1, 'upper')
(53, 'Harper, Mrs. Henry Sleeper (Myna Haxtun)', 1, 49

In [106]:
get_by_class(["lower","upper"])

(1, 'Braund, Mr. Owen Harris', 0, 22, 0, 3, 1, 0, 'A/5 21171', 7.25, 'S', 3, 'lower')
(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 1, 38, 1, 1, 1, 0, 'PC 17599', 71.28, 'C', 1, 'upper')
(3, 'Heikkinen, Miss. Laina', 1, 26, 1, 3, 0, 0, 'STON/O2. 3101282', 7.92, 'S', 3, 'lower')
(4, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 1, 35, 1, 1, 1, 0, '113803', 53.1, 'S', 1, 'upper')
(5, 'Allen, Mr. William Henry', 0, 35, 0, 3, 0, 0, '373450', 8.05, 'S', 3, 'lower')
(6, 'Moran, Mr. James', 0, None, 0, 3, 0, 0, '330877', 8.46, 'Q', 3, 'lower')
(7, 'McCarthy, Mr. Timothy J', 0, 54, 0, 1, 0, 0, '17463', 51.86, 'S', 1, 'upper')
(8, 'Palsson, Master. Gosta Leonard', 0, 2, 0, 3, 3, 1, '349909', 21.08, 'S', 3, 'lower')
(9, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 1, 27, 1, 3, 0, 2, '347742', 11.13, 'S', 3, 'lower')
(11, 'Sandstrom, Miss. Marguerite Rut', 1, 4, 1, 3, 1, 1, 'PP 9549', 16.7, 'S', 3, 'lower')
(12, 'Bonnell, Miss. Elizabeth', 1, 58, 1, 1, 0, 0, '113783', 26.55

#### Restrict By Sex

In [111]:
def get_by_sex(sex):
    d = {"M": 0, "F": 1}
    for row in c.execute(f'''
        SELECT *
        FROM Passenger WHERE sex = {d[sex]}
        '''):
            print(row)

In [112]:
get_by_sex("M")

(1, 'Braund, Mr. Owen Harris', 0, 22, 0, 3, 1, 0, 'A/5 21171', 7.25, 'S')
(5, 'Allen, Mr. William Henry', 0, 35, 0, 3, 0, 0, '373450', 8.05, 'S')
(6, 'Moran, Mr. James', 0, None, 0, 3, 0, 0, '330877', 8.46, 'Q')
(7, 'McCarthy, Mr. Timothy J', 0, 54, 0, 1, 0, 0, '17463', 51.86, 'S')
(8, 'Palsson, Master. Gosta Leonard', 0, 2, 0, 3, 3, 1, '349909', 21.08, 'S')
(13, 'Saundercock, Mr. William Henry', 0, 20, 0, 3, 0, 0, 'A/5. 2151', 8.05, 'S')
(14, 'Andersson, Mr. Anders Johan', 0, 39, 0, 3, 1, 5, '347082', 31.28, 'S')
(17, 'Rice, Master. Eugene', 0, 2, 0, 3, 4, 1, '382652', 29.12, 'Q')
(18, 'Williams, Mr. Charles Eugene', 0, None, 1, 2, 0, 0, '244373', 13.0, 'S')
(21, 'Fynney, Mr. Joseph J', 0, 35, 0, 2, 0, 0, '239865', 26.0, 'S')
(22, 'Beesley, Mr. Lawrence', 0, 34, 1, 2, 0, 0, '248698', 13.0, 'S')
(24, 'Sloper, Mr. William Thompson', 0, 28, 1, 1, 0, 0, '113788', 35.5, 'S')
(27, 'Emir, Mr. Farred Chehab', 0, None, 0, 3, 0, 0, '2631', 7.22, 'C')
(28, 'Fortune, Mr. Charles Alexander', 0, 19

In [114]:
get_by_sex("F")

(2, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 1, 38, 1, 1, 1, 0, 'PC 17599', 71.28, 'C')
(3, 'Heikkinen, Miss. Laina', 1, 26, 1, 3, 0, 0, 'STON/O2. 3101282', 7.92, 'S')
(4, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 1, 35, 1, 1, 1, 0, '113803', 53.1, 'S')
(9, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 1, 27, 1, 3, 0, 2, '347742', 11.13, 'S')
(10, 'Nasser, Mrs. Nicholas (Adele Achem)', 1, 14, 1, 2, 1, 0, '237736', 30.07, 'C')
(11, 'Sandstrom, Miss. Marguerite Rut', 1, 4, 1, 3, 1, 1, 'PP 9549', 16.7, 'S')
(12, 'Bonnell, Miss. Elizabeth', 1, 58, 1, 1, 0, 0, '113783', 26.55, 'S')
(15, 'Vestrom, Miss. Hulda Amanda Adolfina', 1, 14, 0, 3, 0, 0, '350406', 7.85, 'S')
(16, 'Hewlett, Mrs. (Mary D Kingcome) ', 1, 55, 1, 2, 0, 0, '248706', 16.0, 'S')
(19, 'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)', 1, 31, 0, 3, 1, 0, '345763', 18.0, 'S')
(20, 'Masselmani, Mrs. Fatima', 1, None, 1, 3, 0, 0, '2649', 7.22, 'C')
(23, 'McGowan, Miss. Anna "Annie"', 1, 15, 1, 3